In [1]:
# подгружаем библиотеки
import numpy as np
import pandas as pd
import requests
from tqdm import tqdm
from neo4j import GraphDatabase # pip install neo4j

In [7]:
# подключения питона к neo4j
class Neo4jConnection:
    def __init__(self, uri, user, password):
        self.driver = GraphDatabase.driver(uri, auth=(user, password))

    def close(self):
        if self.driver is not None:
            self.driver.close()

# Метод, который передает запрос в БД
    def query(self, query, db=None):
        assert self.driver is not None, "Driver not initialized!"
        session = None
        response = None
        try:
            session = self.driver.session(database=db) if db is not None else self.driver.session()
            response = list(session.run(query))
        except Exception as e:
            print("Query failed:", e)
        finally:
            if session is not None:
                session.close()
        return response
    
conn = Neo4jConnection(uri="bolt://localhost:7687", user="Ksenia", password="Ct358Bio")
conn.query("CREATE OR REPLACE DATABASE moviebd")

[]

In [3]:
l = 300 # первый film_id
n = 10 # кол-во индексов перебрать

# парсим фильмы
films = []
for i in tqdm(range(l,l+n)):
    url = 'https://kinopoiskapiunofficial.tech/api/v2.2/films/' + str(i)
    headers = {'X-API-KEY':'883331c9-daee-4173-bbe5-332e13a5252d', 'Content-Type':'application/json',}
    response = requests.get(url, headers = headers)
    if response.status_code == 200:
        films.append(response.json())
# создаем csv
films_1 = pd.DataFrame(films)[['kinopoiskId','nameRu','ratingKinopoisk','ratingKinopoiskVoteCount','year','filmLength','slogan','ratingAgeLimits','shortDescription','countries','genres']]
films_1['countries'] = films_1['countries'].map(lambda x: [item['country'] for item in x])
films_1['genres'] = films_1['genres'].map(lambda x: [item['genre'] for item in x])
films_1.to_csv('C:/Users/79653/.Neo4jDesktop/relate-data/dbmss/dbms-2d2b424a-9a21-41ab-8dd3-d3640660c98a/import/films.csv', index = None)

# парсим людей
persons = []
for i in tqdm(range(l,l+n)):
    url = 'https://kinopoiskapiunofficial.tech/api/v1/staff?filmId=' + str(i)
    headers = {'X-API-KEY':'883331c9-daee-4173-bbe5-332e13a5252d', 'Content-Type':'application/json',}
    response = requests.get(url, headers = headers)
    if response.status_code == 200:
        persons.append(response.json())
# создаем csv
k = []
for i in persons:
    for j in i:
        k.append(j)
persons_1 = pd.DataFrame(k)[['staffId','nameRu']].drop_duplicates()
persons_1.to_csv('C:/Users/79653/.Neo4jDesktop/relate-data/dbmss/dbms-2d2b424a-9a21-41ab-8dd3-d3640660c98a/import/persons.csv', index = None)

# создаем вспомогательную таблицу (для связей узлов)
film_id = []
staff_id = []
proff_key = []
roles = []
for i in range(len(pd.Series(films).map(lambda x: x['kinopoiskId']))):
    for j in persons[i]:
        film_id.append(pd.Series(films).map(lambda x: x['kinopoiskId']).iloc[i])
        staff_id.append(j['staffId'])
        proff_key.append(j['professionKey'])
        roles.append(j['description'])
extra_table = pd.DataFrame({'film_id':film_id,'staff_id':staff_id,'proff_key':proff_key,'roles':roles})
extra_table = extra_table[extra_table['proff_key'].isin(['DIRECTOR','ACTOR','PRODUCER','WRITER'])] # оставляем только нужные связи
extra_table = extra_table.drop(extra_table[extra_table.proff_key == 'ACTOR'][extra_table[extra_table.proff_key == 'ACTOR']['roles'].isna()].index) # оставляем только актеров, у которых указана роль
# создаем csv
extra_table[extra_table['proff_key'] == 'ACTOR'].to_csv('C:/Users/79653/.Neo4jDesktop/relate-data/dbmss/dbms-2d2b424a-9a21-41ab-8dd3-d3640660c98a/import/extra_table_actors.csv', index = None)
extra_table[extra_table['proff_key'] == 'PRODUCER'].to_csv('C:/Users/79653/.Neo4jDesktop/relate-data/dbmss/dbms-2d2b424a-9a21-41ab-8dd3-d3640660c98a/import/extra_table_producers.csv', index = None)
extra_table[extra_table['proff_key'] == 'WRITER'].to_csv('C:/Users/79653/.Neo4jDesktop/relate-data/dbmss/dbms-2d2b424a-9a21-41ab-8dd3-d3640660c98a/import/extra_table_writers.csv', index = None)
extra_table[extra_table['proff_key'] == 'DIRECTOR'].to_csv('C:/Users/79653/.Neo4jDesktop/relate-data/dbmss/dbms-2d2b424a-9a21-41ab-8dd3-d3640660c98a/import/extra_table_directors.csv', index = None)

100%|██████████████████████████████████████████████████████████████████████████████████| 10/10 [00:00<00:00, 14.62it/s]


In [8]:
# загружаем узлы (люди)
query_string = '''
LOAD CSV WITH HEADERS FROM
'file:///persons.csv'
AS persons FIELDTERMINATOR ','
MERGE (person:Person {personID: persons.staffId})
  ON CREATE SET person.Name = persons.nameRu;
'''
conn.query(query_string, db='moviebd')

[]

In [9]:
# загружаем узлы (фильмы)
query_string = '''
LOAD CSV WITH HEADERS FROM
'file:///films.csv'
AS films FIELDTERMINATOR ','
MERGE (film:Movie {filmID: films.kinopoiskId})
  ON CREATE SET film.Name = films.nameRu, film.Rating = films.ratingKinopoisk, film.VoteCount = films.ratingKinopoiskVoteCount, film.Year = films.yesr, film.Length = films.filmLength, film.Slogan = films.slogan, film.AgeLimits = films.ratingAgeLimits, film.Description = films.shortDescription, film.Countries = films.countries, film.Genres = films.genres;
'''
conn.query(query_string, db='moviebd')

[]

In [10]:
# загружаем связи (люди -> фильмы)
# актеры:
query_string = '''
LOAD CSV WITH HEADERS FROM 'file:///extra_table_actors.csv' AS line
MATCH (person:Person {personID: line.staff_id})
MATCH (film:Movie {filmID: line.film_id})
MERGE (person)-[op:ACTED_IN]->(film)
  ON CREATE SET op.roles = line.roles;
'''
conn.query(query_string, db='moviebd')

# продюсеры:
query_string ='''
LOAD CSV WITH HEADERS FROM 'file:///extra_table_producers.csv' AS line
MATCH (person:Person {personID: line.staff_id})
MATCH (film:Movie {filmID: line.film_id})
CREATE (person)-[:PRODUCED]->(film);
'''
conn.query(query_string, db='moviebd')

# режиссеры:
query_string ='''
LOAD CSV WITH HEADERS FROM 'file:///extra_table_directors.csv' AS line
MATCH (person:Person {personID: line.staff_id})
MATCH (film:Movie {filmID: line.film_id})
CREATE (person)-[:DIRECTED]->(film);
'''
conn.query(query_string, db='moviebd')

# писатели:
query_string ='''
LOAD CSV WITH HEADERS FROM 'file:///extra_table_writers.csv' AS line
MATCH (person:Person {personID: line.staff_id})
MATCH (film:Movie {filmID: line.film_id})
CREATE (person)-[:WRITTEN]->(film);
'''
conn.query(query_string, db='moviebd')

[]